# SLSM Wrapper - Local Environment

This notebook runs the SLSM (Semantic-Level State Machine) wrapper locally.

**Prerequisites:**
1. Activate conda environment: `conda activate multichallenge`
2. Create `.env` file in project root with `OPENAI_API_KEY=sk-...`

In [11]:
# =========================
# Environment Setup (Local)
# =========================
import os
import sys
from pathlib import Path

# Set project root
PROJECT_ROOT = Path("/home/wenhel/multi-challenge")
os.chdir(PROJECT_ROOT)
sys.path.insert(0, str(PROJECT_ROOT))

print(f"Working directory: {os.getcwd()}")
print(f"Python path includes: {PROJECT_ROOT}")

Working directory: /home/wenhel/multi-challenge
Python path includes: /home/wenhel/multi-challenge


In [13]:
# =========================
# Load API Keys from .env
# =========================
from dotenv import load_dotenv

# Load from .env file in project root
env_path = PROJECT_ROOT / ".env"
if env_path.exists():
    load_dotenv(env_path)
    print(f"Loaded environment from: {env_path}")
else:
    print(f"WARNING: .env file not found at {env_path}")
    print("Please create .env with OPENAI_API_KEY=sk-...")

# Verify API key is set
api_key = os.environ.get("OPENAI_API_KEY", "")
if api_key:
    print(f"OPENAI_API_KEY is set (starts with: {api_key[:8]}...)")
else:
    raise ValueError("OPENAI_API_KEY not found in environment")

Loaded environment from: /home/wenhel/multi-challenge/.env
OPENAI_API_KEY is set (starts with: sk-proj-...)


In [ ]:
# =========================
# Install/Check Dependencies
# =========================
# !pip install -q -r requirements.txt

## Load Benchmark Data

In [14]:
from src.data_loader import DataLoader

BENCHMARK = "data/benchmark_questions.jsonl"

dl = DataLoader(input_file=BENCHMARK)
dl.load_data()

conversations = dl.get_conversations()

print(f"Loaded {len(conversations)} conversations")
print(f"Type: {type(conversations[0])}")

Loaded 273 conversations
Type: <class 'src.conversation.Conversation'>


In [15]:
# Inspect first conversation structure
conv = conversations[0]
messages = conv.conversation  # list[{"role", "content"}]

print(f"Keys in message: {messages[0].keys()}")
print(f"Number of turns: {len(messages)}")

Keys in message: dict_keys(['role', 'content'])
Number of turns: 3


## SLSM Implementation Test

In [16]:
# =========================
# SLSM Sanity Check
# =========================

from src.data_loader import DataLoader
from src.models.openai import OpenAIModel
from src.slsm_wrapper import (
    SLSMConfig,
    SLSMController,
    SLSMWrapper,
)

# Load conversations
BENCHMARK = "data/benchmark_questions.jsonl"
dl = DataLoader(input_file=BENCHMARK)
dl.load_data()
conversations = dl.get_conversations()

# Pick one conversation
conv = conversations[0]
messages = conv.conversation

# Print full conversation (indexed)
print("=== FULL CONVERSATION (indexed) ===")
for i, m in enumerate(messages):
    role = m.get("role")
    content = (m.get("content") or "")
    # Truncate long turns for readability
    preview = content if len(content) <= 500 else content[:500] + " ...[truncated]"
    print(f"\n--- turn {i} | {role} ---\n{preview}")

=== FULL CONVERSATION (indexed) ===

--- turn 0 | user ---
Hello!  I am an International relations expert working at the UN headquarters. My work requires me to consistently meet with diplomats from various countries. I hate using taxis or public transportation in New York. I prefer venues that are within a 5-minute walk from the UN headquarters.

--- turn 1 | assistant ---
Hello! It's great to hear that you're looking for places near the UN headquarters in New York. Here are a few suggestions that are within a 5-minute walk:

1. Dag Hammarskjold Plaza: This public park is just across the street from the UN headquarters. It's a great place for a peaceful walk or a quick meeting.

2. The Delegates Dining Room: Located within the UN headquarters itself, this is a convenient place for a meeting over lunch or dinner.

3. The Roosevelt Hotel: This historic hotel is just ...[truncated]

--- turn 2 | user ---
 I am meeting a German diplomat on Friday. I am looking for a suitable place to have

In [17]:
# =========================
# Initialize SLSM Components
# =========================

# Controller (cheap, fixed temperature)
controller_llm = OpenAIModel(
    model="gpt-4o-mini",
    temp=0
)

cfg = SLSMConfig(
    inject="always",   # Force injection for testing
    note_max_items=6,
)

controller = SLSMController(controller_llm, cfg)
wrapper = SLSMWrapper(controller, cfg)

# Underlying model (tested model)
underlying_llm = OpenAIModel(
    model="gpt-4o-2024-08-06",
    temp=0
)

print("SLSM components initialized.")

SLSM components initialized.


In [18]:
# =========================
# Compare Baseline vs SLSM
# =========================

# Baseline response
baseline_resp = underlying_llm.generate(messages)

# SLSM wrapped response
slsm_resp = wrapper.generate_last_turn(
    underlying_llm=underlying_llm,
    original_conversation=messages,
)

print("\n=== BASELINE ===")
print(baseline_resp[:500])

print("\n=== SLSM ===")
print(slsm_resp[:500])


=== BASELINE ===
Certainly! Here are a few upscale restaurants near the UN headquarters that would be suitable for a lunch meeting with a German diplomat:

1. **The Modern**: Located at the Museum of Modern Art, The Modern offers a refined dining experience with contemporary American cuisine. It's a bit further than a 5-minute walk, but it's a top choice for an upscale meal.

2. **Aquavit**: This Michelin-starred restaurant offers Scandinavian cuisine and is known for its elegant setting and exceptional service.

=== SLSM ===
Certainly! Here are some up-class restaurants within a 5-minute walk from the UN headquarters where you can have lunch with the German diplomat:

1. **The Modern**: Located at the Museum of Modern Art, this Michelin-starred restaurant offers a refined dining experience with contemporary American cuisine. It's a bit of a walk but still within a reasonable distance.

2. **Aquavit**: This two-Michelin-starred restaurant offers a sophisticated dining experience with 

In [19]:
# =========================
# Inspect SLSM State
# =========================

# Inspect state and injected note
state = wrapper.track_state(messages)
print("\n=== RAW STATE FACTS (controller output, pre-gating) ===")
print(state.facts)

msgs = wrapper.build_final_messages(messages, state)
print("\n=== FINAL MSG ROLES (first 6) ===")
print([m["role"] for m in msgs[:6]])

print("\n=== INJECTED NOTE (first 1200 chars) ===")
print(msgs[0]["role"], msgs[0]["content"][:1200])


=== RAW STATE FACTS (controller output, pre-gating) ===
[{'id': 'F1', 'text': 'User is an International relations expert working at the UN headquarters.', 'support_turns': [0], 'evidence': 'I am an International relations expert working at the UN headquarters.'}]

=== FINAL MSG ROLES (first 6) ===
['system', 'user', 'assistant', 'user']

=== INJECTED NOTE (first 1200 chars) ===
system [SLSM MEMORY NOTE]
Constraints:
- [satisfied] User prefers venues that are within a 5-minute walk from the UN headquarters.
- [satisfied] User hates using taxis or public transportation in New York.
- [satisfied] User is looking for an up-class restaurant for lunch with a German diplomat.
- [satisfied] User does not need other arrangements, particularly security arrangements.
User facts/preferences:
- User is an International relations expert working at the UN headquarters.

Follow the constraints above.


## Run SLSM on Full Benchmark (First 50 Samples)

In [21]:
# =========================
# Run SLSM on Benchmark
# =========================

!/home/wenhel/miniconda3/envs/multichallenge/bin/python run_slsm_multichallenge_gpt4o.py

Loaded 273 conversations
Running SLSM-controlled GPT-4o: 100%|█████████| 50/50 [1:03:41<00:00, 76.43s/it]

Done. Results saved to:
  data/final_model_responses/gpt-4o-2024-08-06_slsm-gpt-4o-mini-v2.jsonl


In [23]:
# =========================
# Prepare Baseline First 50
# =========================
import json

src = "data/final_model_responses/gpt-4o-2024-08-06_slsm-gpt-4o-mini-v2.jsonl"
dst = "data/final_model_responses/gpt-4o-2024-08-06_slsm-gpt-4o-mini-v2_responses_first50.jsonl"

with open(src, "r", encoding="utf-8") as f:
    lines = [next(f) for _ in range(50)]

with open(dst, "w", encoding="utf-8") as g:
    for line in lines:
        g.write(line)

print(f"Wrote: {dst} lines={len(lines)}")

Wrote: data/final_model_responses/gpt-4o-2024-08-06_slsm-gpt-4o-mini-v2_responses_first50.jsonl lines=50


## Judge Evaluation

In [25]:
# =========================
# Evaluate Baseline
# =========================
import os
os.makedirs("outputs_first50", exist_ok=True)

!/home/wenhel/miniconda3/envs/multichallenge/bin/python -m run_judge_eval \
  --responses data/final_model_responses/gpt-4o-2024-08-06_slsm-gpt-4o-mini-v2_responses_first50.jsonl \
  --out_json outputs_first50/gpt4o_baseline_judge_results.json \
  --out_csv outputs_first50/gpt4o_baseline_judge_results.csv \
  --workers 1 \
  --attempts 1

print("Done. Baseline judge evaluation saved to outputs_first50/")

Evaluating responses: 100%|█████████████████████| 50/50 [01:51<00:00,  2.22s/it]

=== SCORES ===
{
  "overall_score": 3.863828176565721,
  "axis_scores": {
    "INFERENCE_MEMORY": 1.7699115044247788,
    "RELIABLE_VERSION_EDITING": 2.4390243902439024,
    "SELF_COHERENCE": 4.0,
    "INSTRUCTION_RETENTION": 7.246376811594203
  }
}

Saved: outputs_first50/gpt4o_baseline_judge_results.json
Saved: outputs_first50/gpt4o_baseline_judge_results.csv
Done. Baseline judge evaluation saved to outputs_first50/


In [ ]:
# =========================
# Fix SLSM Response Format
# =========================
import json

src = "data/final_model_responses/gpt-4o-2024-08-06_slsm-gpt-4o-mini.jsonl"
dst = "data/final_model_responses/gpt-4o-2024-08-06_slsm-gpt-4o-mini_mcformat.jsonl"

n = 0
with open(src, "r", encoding="utf-8") as f, open(dst, "w", encoding="utf-8") as g:
    for line in f:
        obj = json.loads(line)

        # Normalize keys
        qid = obj.get("QUESTION_ID", obj.get("question_id", obj.get("qid")))
        if qid is None:
            raise KeyError(f"Missing question id in line: {obj.keys()}")

        resp = obj.get("RESPONSE", obj.get("response", obj.get("answer")))
        if resp is None:
            raise KeyError(f"Missing response text in line: {obj.keys()}")

        model = obj.get("MODEL", obj.get("model", "UNKNOWN_MODEL"))

        out = {
            "QUESTION_ID": qid,
            "MODEL": model,
            "RESPONSE": resp,
        }
        g.write(json.dumps(out, ensure_ascii=False) + "\n")
        n += 1

print(f"Wrote {n} lines -> {dst}")

In [ ]:
# =========================
# Evaluate SLSM
# =========================

!python -m run_judge_eval \
  --responses data/final_model_responses/gpt-4o-2024-08-06_slsm-gpt-4o-mini_mcformat.jsonl \
  --out_json outputs_first50/gpt4o_slsm_judge_results.json \
  --out_csv outputs_first50/gpt4o_slsm_judge_results.csv \
  --workers 1 \
  --attempts 1

print("Done. SLSM judge results saved to outputs_first50/")

## Compare Results: Baseline vs SLSM

In [ ]:
# =========================
# Compare Judge Results
# =========================

import pandas as pd
import numpy as np
from pathlib import Path

# Configure paths
BASE_CSV = Path("outputs_first50/gpt4o_baseline_judge_results.csv")
SLSM_CSV = Path("outputs_first50/gpt4o_slsm_judge_results.csv")

if not BASE_CSV.exists() or not SLSM_CSV.exists():
    print("Missing CSV files. Run evaluation cells first.")
else:
    base = pd.read_csv(BASE_CSV)
    slsm = pd.read_csv(SLSM_CSV)

    print(f"Baseline CSV: {BASE_CSV} | rows: {len(base)} | cols: {len(base.columns)}")
    print(f"SLSM CSV    : {SLSM_CSV} | rows: {len(slsm)} | cols: {len(slsm.columns)}")

In [ ]:
# =========================
# Bootstrap CI Analysis
# =========================

# Find ID column
id_candidates = ["QUESTION_ID", "question_id", "qid", "id"]
id_col = None
for c in id_candidates:
    if c in base.columns and c in slsm.columns:
        id_col = c
        break

if id_col is None:
    commons = set(base.columns) & set(slsm.columns)
    for c in commons:
        if "id" in c.lower():
            id_col = c
            break

assert id_col is not None, f"Cannot find common ID column. baseline cols={list(base.columns)}"

# Merge
df = base.merge(slsm, on=id_col, how="inner", suffixes=("_base", "_slsm"))
assert len(df) > 0, "Merged dataframe is empty"
print(f"ID column: {id_col} | merged rows: {len(df)}")

# Detect metric pairs
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
num_cols = [c for c in num_cols if c != id_col]

pairs = []
for c in num_cols:
    if c.endswith("_base"):
        c2 = c[:-5] + "_slsm"
        if c2 in df.columns:
            pairs.append((c, c2))

print(f"Found {len(pairs)} metric pairs")

In [ ]:
# =========================
# Calculate Metrics
# =========================

rng = np.random.default_rng(0)

def bootstrap_ci_mean_diff(x, y, n_boot=5000, alpha=0.05):
    """
    Returns (mean_diff, ci_low, ci_high) for (y-x) using bootstrap.
    """
    diffs = (y - x).astype(float)
    n = diffs.shape[0]
    idx = rng.integers(0, n, size=(n_boot, n))
    boot_means = diffs[idx].mean(axis=1)
    lo = np.quantile(boot_means, alpha/2)
    hi = np.quantile(boot_means, 1 - alpha/2)
    return float(diffs.mean()), float(lo), float(hi)

rows = []
for b, s in pairs:
    metric = b[:-5]  # remove "_base"
    x = df[b].to_numpy()
    y = df[s].to_numpy()

    base_mean = float(np.mean(x))
    slsm_mean = float(np.mean(y))
    dmean, dlo, dhi = bootstrap_ci_mean_diff(x, y)

    win = float(np.mean(y > x))
    tie = float(np.mean(y == x))
    lose = float(np.mean(y < x))

    rows.append({
        "metric": metric,
        "baseline_mean": base_mean,
        "slsm_mean": slsm_mean,
        "delta_mean": dmean,
        "delta_ci_low": dlo,
        "delta_ci_high": dhi,
        "win_rate": win,
        "tie_rate": tie,
        "lose_rate": lose,
        "n": int(len(x)),
    })

summary = pd.DataFrame(rows).sort_values("metric").reset_index(drop=True)

print("\n=== METRIC SUMMARY ===")
display(summary)

In [ ]:
# =========================
# Generate LaTeX Table
# =========================

def fmt(x, nd=3):
    return f"{x:.{nd}f}"

latex = []
latex += [r"\begin{table}[t]",
          r"\centering",
          r"\small",
          r"\begin{tabular}{lrrrr}",
          r"\toprule",
          r"Metric & Baseline & SLSM & $\Delta$ (boot 95\% CI) & Win-rate \\",
          r"\midrule"]

for _, r in summary.iterrows():
    metric = r["metric"]
    base_m = fmt(r["baseline_mean"])
    slsm_m = fmt(r["slsm_mean"])
    d = fmt(r["delta_mean"])
    lo = fmt(r["delta_ci_low"])
    hi = fmt(r["delta_ci_high"])
    win = f"{100*r['win_rate']:.1f}" + r"\%"
    latex.append(f"{metric} & {base_m} & {slsm_m} & {d} [{lo}, {hi}] & {win} \\\\")

latex += [r"\bottomrule",
          r"\end{tabular}",
          r"\caption{Judge scores: baseline GPT-4o vs SLSM-controlled GPT-4o}",
          r"\label{tab:mc_judge}",
          r"\end{table}"]

latex_str = "\n".join(latex)
print("\n=== LaTeX TABLE ===\n")
print(latex_str)

In [ ]:
# =========================
# Save Results
# =========================

out_dir = Path("outputs_first50")
out_dir.mkdir(parents=True, exist_ok=True)

summary_path = out_dir / "baseline_vs_slsm_summary_first50.csv"
tex_path = out_dir / "baseline_vs_slsm_table_first50.tex"

summary.to_csv(summary_path, index=False)
tex_path.write_text(latex_str, encoding="utf-8")

print(f"Saved summary CSV: {summary_path}")
print(f"Saved LaTeX table: {tex_path}")